In order to run this notebook with the correct PYTHONPATH, you can use
```
$ ./e-mission-jupyter.bash notebook
```

This notebook makes heavy use of the following python libraries.
- `pandas`: from the python computing stack (http://pandas.pydata.org/pandas-docs/stable/)
- `geojson`: standard JSON representation of geographic data (http://geojson.org/)
- `folium`: (https://github.com/python-visualization/folium) python bridge to leaflet (http://leafletjs.com/)

### Pick a user to work with ###

In [7]:
import emission.core.get_database as edb
import pandas as pd
all_users = pd.DataFrame(list(edb.get_uuid_db().find()))
# all_users = pd.DataFrame(list(edb.get_uuid_db().find({}, {"user_email":1, "uuid": 1, "_id": 0})))
all_users

Connecting to database URL localhost


InterfaceError: 2003: Can't connect to MySQL server on 'cryptdb:3307' (8 nodename nor servname provided, or not known)

In [2]:
test = pd.DataFrame(list(edb.get_profile_db().find()))
test

,_id,mpg_array,source,update_ts,user_id
0,5bf5904a6f0a85cfea3d42bf,[32.044384997811726],Shankari,2018-11-21 10:24:29.488,6a1717a2-99f9-4051-bb7a-d9c06f9287fa


In [11]:
test.dtypes

_id         object
data        object
metadata    object
user_id     object
dtype: object

In [32]:
print(test.loc([test._id=="5bf58c3236a3f504d4803d8d"])[0])

_id                                  5bbaf141e7b8de39f8f68ee5
data        {'name': 'GET_/', 'ts': 1538978113.892245, 're...
metadata    {'key': 'stats/server_api_time', 'platform': '...
user_id                                                      
Name: 0, dtype: object


In [3]:
from uuid import UUID

In [4]:
test_user_id = all_users.iloc[0].uuid # replace with UUID from above
test_user_id

UUID('6a1717a2-99f9-4051-bb7a-d9c06f9287fa')

If you want to work across multiple users, just do the same thing again

In [5]:
test_user_id_2 = UUID('0763de67f61e3f5d90e7518e69793954')
test_user_id_2

UUID('0763de67-f61e-3f5d-90e7-518e69793954')

### Preferred access technique

The preferred technique to access wrapper objects from the timeseries is to use the abstract timeseries interface. This makes it easier for us to switch to alternative timeseries implementations later. The timeseries is conceptually a set of streams, one for each of the types, primarily indexed by time. So you can query for all entries of a particular time within a specified time range.

In [5]:
import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.decorations.analysis_timeseries_queries as esda
import emission.core.wrapper.entry as ecwe
#import emission.storage.decorations.trip_queries as esdt
import emission.storage.timeseries.timequery as estt

In [6]:
ts = esta.TimeSeries.get_time_series(test_user_id)

Connecting to database URL localhost


InterfaceError: 2003: Can't connect to MySQL server on 'cryptdb:3307' (8 nodename nor servname provided, or not known)

In [7]:
print(test_user_id, test_user_id_2)
ts_2 = esta.TimeSeries.get_time_series(test_user_id_2)
print(ts)

NameError: name 'test_user_id_2' is not defined

#### Accessing entries directly

In [11]:
# Get all cleaned trips for the first user
entry_it = ts.find_entries(["analysis/cleaned_trip"], time_query=None)

In [12]:
for ct in entry_it:
    cte = ecwe.Entry(ct)
    print("=== Trip:", cte.data.start_loc, "->", cte.data.end_loc)
    section_it = esdt.get_sections_for_trip("analysis/cleaned_section", test_user_id, cte.get_id())
    for sec in section_it:
        print("  --- Section:", sec.data.start_loc, "->", sec.data.end_loc, " on ", sec.data.sensed_mode)

=== Trip: {"coordinates": [-122.0876886, 37.3887767], "type": "Point"} -> {"coordinates": [-122.0820411, 37.3920436], "type": "Point"}
  --- Section: {"coordinates": [-122.0876886, 37.3887767], "type": "Point"} -> {"coordinates": [-122.0773235, 37.3946683], "type": "Point"}  on  MotionTypes.BICYCLING
  --- Section: {"coordinates": [-122.077133, 37.3944385], "type": "Point"} -> {"coordinates": [-122.0859763, 37.3983699], "type": "Point"}  on  MotionTypes.ON_FOOT
  --- Section: {"coordinates": [-122.1031161, 37.4048536], "type": "Point"} -> {"coordinates": [-122.1651563, 37.4438859], "type": "Point"}  on  MotionTypes.IN_VEHICLE
  --- Section: {"coordinates": [-122.1647388, 37.4438775], "type": "Point"} -> {"coordinates": [-122.1631988, 37.4459894], "type": "Point"}  on  MotionTypes.ON_FOOT
  --- Section: {"coordinates": [-122.1631156, 37.4464546], "type": "Point"} -> {"coordinates": [-122.1658293, 37.4438399], "type": "Point"}  on  MotionTypes.BICYCLING
  --- Section: {"coordinates": [-1

All keys and their mapping to data model objects can be found in 
https://github.com/e-mission/e-mission-server/blob/master/emission/core/wrapper/entry.py

In [18]:
for ct in entry_it:
    cte = ecwe.Entry(ct)
    print("=== Trip:", cte.data.start_loc, "->", cte.data.end_loc)
    section_it = esdt.get_sections_for_trip("analysis/cleaned_section", test_user_id, cte.get_id())
    for sec in section_it:
        print("  --- Section:", sec.data.start_loc, "->", sec.data.end_loc, " on ", sec.data.sensed_mode)

=== Trip: {"coordinates": [-122.0876886, 37.3887767], "type": "Point"} -> {"coordinates": [-122.0820411, 37.3920436], "type": "Point"}
  --- Section: {"coordinates": [-122.0876886, 37.3887767], "type": "Point"} -> {"coordinates": [-122.0773235, 37.3946683], "type": "Point"}  on  MotionTypes.BICYCLING
  --- Section: {"coordinates": [-122.077133, 37.3944385], "type": "Point"} -> {"coordinates": [-122.0859763, 37.3983699], "type": "Point"}  on  MotionTypes.ON_FOOT
  --- Section: {"coordinates": [-122.1031161, 37.4048536], "type": "Point"} -> {"coordinates": [-122.1651563, 37.4438859], "type": "Point"}  on  MotionTypes.IN_VEHICLE
  --- Section: {"coordinates": [-122.1647388, 37.4438775], "type": "Point"} -> {"coordinates": [-122.1631988, 37.4459894], "type": "Point"}  on  MotionTypes.ON_FOOT
  --- Section: {"coordinates": [-122.1631156, 37.4464546], "type": "Point"} -> {"coordinates": [-122.1658293, 37.4438399], "type": "Point"}  on  MotionTypes.BICYCLING
  --- Section: {"coordinates": [-1

In [13]:
# Get all cleaned trips for the second user
entry_it = ts_2.find_entries(["analysis/cleaned_trip"], time_query=None)

In [14]:
for ct in entry_it:
    cte = ecwe.Entry(ct)
    print("=== Trip:", cte.data.start_loc, "->", cte.data.end_loc)
    section_it = esdt.get_sections_for_trip("analysis/cleaned_section", test_user_id, cte.get_id())
    for sec in section_it:
        print("  --- Section:", sec.data.start_loc, "->", sec.data.end_loc, " on ", sec.data.sensed_mode)

In [15]:
# Get cleaned trips for the two users that started on 1st Aug UTC
import arrow

aug_1_tq = estt.TimeQuery("data.start_ts",
                          arrow.get("2017-08-01").timestamp, # start of range
                          arrow.get("2017-08-02").timestamp)  # end of range
entry_it = ts.find_entries(["analysis/cleaned_trip"], time_query=aug_1_tq)
entry_it_2 = ts_2.find_entries(["analysis/cleaned_trip"], time_query=aug_1_tq)
print("From %s -> %s, user %s had %d trips and user %s had %d trips" %
     (aug_1_tq.startTs, aug_1_tq.endTs, test_user_id, len(list(entry_it)), test_user_id_2, len(list(entry_it_2))))

From 1501545600 -> 1501632000, user de0f61b9-8e0d-45f0-8c64-09e6fbabd7e9 had 0 trips and user 0763de67-f61e-3f5d-90e7-518e69793954 had 0 trips


#### Accessing a dataframe

In [16]:
# Get all cleaned trips for the first user
ct_df = ts.get_data_df("analysis/cleaned_trip", time_query=None)

In [23]:
len(ct_df)

6

In [17]:
ct_df.columns

Index(['_id', 'distance', 'duration', 'end_fmt_time', 'end_loc',
       'end_local_dt_day', 'end_local_dt_hour', 'end_local_dt_minute',
       'end_local_dt_month', 'end_local_dt_second', 'end_local_dt_timezone',
       'end_local_dt_weekday', 'end_local_dt_year', 'end_place', 'end_ts',
       'metadata_write_ts', 'raw_trip', 'source', 'start_fmt_time',
       'start_loc', 'start_local_dt_day', 'start_local_dt_hour',
       'start_local_dt_minute', 'start_local_dt_month',
       'start_local_dt_second', 'start_local_dt_timezone',
       'start_local_dt_weekday', 'start_local_dt_year', 'start_place',
       'start_ts', 'user_id'],
      dtype='object')

In [25]:
ct_df[["start_loc", "end_loc", "start_fmt_time", "end_fmt_time"]]

,start_loc,end_loc,start_fmt_time,end_fmt_time
0,"{'type': 'Point', 'coordinates': [-122.0876886...","{'type': 'Point', 'coordinates': [-122.0820411...",2015-07-22T08:14:53.881000-07:00,2015-07-22T09:23:34.517000-07:00
1,"{'type': 'Point', 'coordinates': [-122.0820411...","{'type': 'Point', 'coordinates': [-122.0862974...",2015-07-22T09:25:15.494000-07:00,2015-07-22T09:30:09.011000-07:00
2,"{'type': 'Point', 'coordinates': [-122.0862974...","{'type': 'Point', 'coordinates': [-122.0801222...",2015-07-22T13:15:24.742251-07:00,2015-07-22T13:22:52.410000-07:00
3,"{'type': 'Point', 'coordinates': [-122.0801222...","{'type': 'Point', 'coordinates': [-122.1592342...",2015-07-22T13:25:45.137266-07:00,2015-07-22T13:53:13.736000-07:00
4,"{'type': 'Point', 'coordinates': [-122.1592342...","{'type': 'Point', 'coordinates': [-122.0823243...",2015-07-22T14:40:47.845961-07:00,2015-07-22T15:39:24.546000-07:00
5,"{'type': 'Point', 'coordinates': [-122.0823243...","{'type': 'Point', 'coordinates': [-122.0862601...",2015-07-22T16:00:26.295595-07:00,2015-07-22T16:28:52.997000-07:00


In [26]:
# Get all cleaned trips for the second user
ct_df_2 = ts_2.get_data_df("analysis/cleaned_trip", time_query=None)
ct_df_2[["start_loc", "end_loc", "start_ts", "end_ts"]]

,start_loc,end_loc,start_ts,end_ts
0,"{'type': 'Point', 'coordinates': [-122.0876886...","{'type': 'Point', 'coordinates': [-122.0820411...",1.437578e+09,1.437582e+09
1,"{'type': 'Point', 'coordinates': [-122.0820411...","{'type': 'Point', 'coordinates': [-122.0862974...",1.437582e+09,1.437583e+09
2,"{'type': 'Point', 'coordinates': [-122.0862974...","{'type': 'Point', 'coordinates': [-122.0801222...",1.437596e+09,1.437597e+09
3,"{'type': 'Point', 'coordinates': [-122.0801222...","{'type': 'Point', 'coordinates': [-122.1592342...",1.437597e+09,1.437598e+09
4,"{'type': 'Point', 'coordinates': [-122.1592342...","{'type': 'Point', 'coordinates': [-122.0823243...",1.437601e+09,1.437605e+09
5,"{'type': 'Point', 'coordinates': [-122.0823243...","{'type': 'Point', 'coordinates': [-122.0862601...",1.437606e+09,1.437608e+09


In [27]:
# Get cleaned trips for the two users that started on 1st Aug UTC
import arrow

aug_1_tq = estt.TimeQuery("data.start_ts",
                          arrow.get("2017-08-01").timestamp, # start of range
                          arrow.get("2017-08-02").timestamp)  # end of range
ct_df = ts.get_data_df("analysis/cleaned_trip", time_query=aug_1_tq)
ct_df_2 = ts_2.get_data_df("analysis/cleaned_trip", time_query=aug_1_tq)
print("From %s -> %s, user %s had %d trips and user %s had %d trips" %
     (aug_1_tq.startTs, aug_1_tq.endTs, test_user_id, len(ct_df), test_user_id_2, len(ct_df_2)))

From 1501545600 -> 1501632000, user de0f61b9-8e0d-45f0-8c64-09e6fbabd7e9 had 0 trips and user de0f61b9-8e0d-45f0-8c64-09e6fbabd7e9 had 0 trips


In [28]:
cs_df = ts.get_data_df("analysis/cleaned_section", time_query=None)

In [29]:
len(cs_df)

21

In [30]:
cs_df.columns

Index(['_id', 'distance', 'distances', 'duration', 'end_fmt_time', 'end_loc',
       'end_local_dt_day', 'end_local_dt_hour', 'end_local_dt_minute',
       'end_local_dt_month', 'end_local_dt_second', 'end_local_dt_timezone',
       'end_local_dt_weekday', 'end_local_dt_year', 'end_stop', 'end_ts',
       'metadata_write_ts', 'sensed_mode', 'source', 'speeds',
       'start_fmt_time', 'start_loc', 'start_local_dt_day',
       'start_local_dt_hour', 'start_local_dt_minute', 'start_local_dt_month',
       'start_local_dt_second', 'start_local_dt_timezone',
       'start_local_dt_weekday', 'start_local_dt_year', 'start_stop',
       'start_ts', 'trip_id', 'user_id'],
      dtype='object')

In [31]:
cs_df[["start_loc", "end_loc", "start_ts", "end_ts", "sensed_mode"]]

,start_loc,end_loc,start_ts,end_ts,sensed_mode
0,"{'type': 'Point', 'coordinates': [-122.0876886...","{'type': 'Point', 'coordinates': [-122.0773235...",1.437578e+09,1.437578e+09,1
1,"{'type': 'Point', 'coordinates': [-122.077133,...","{'type': 'Point', 'coordinates': [-122.0859763...",1.437578e+09,1.437579e+09,2
2,"{'type': 'Point', 'coordinates': [-122.1031161...","{'type': 'Point', 'coordinates': [-122.1651563...",1.437579e+09,1.437580e+09,0
3,"{'type': 'Point', 'coordinates': [-122.1647388...","{'type': 'Point', 'coordinates': [-122.1631988...",1.437580e+09,1.437580e+09,2
4,"{'type': 'Point', 'coordinates': [-122.1631156...","{'type': 'Point', 'coordinates': [-122.1658293...",1.437580e+09,1.437581e+09,1
5,"{'type': 'Point', 'coordinates': [-122.1653719...","{'type': 'Point', 'coordinates': [-122.1559339...",1.437581e+09,1.437581e+09,2
6,"{'type': 'Point', 'coordinates': [-122.1493823...","{'type': 'Point', 'coordinates': [-122.0762984...",1.437581e+09,1.437582e+09,0
7,"{'type': 'Point', 'coordinates': [-122.0767434...","{'type': 'Point', 'coordinates': [-122.0820411...",1.437582e+09,1.437582e+09,2
8,"{'type': 'Point', 'coordinates': [-122.0820411...","{'type': 'Point', 'coordinates': [-122.0862974...",1.437582e+09,1.437583e+09,2
9,"{'type': 'Point', 'coordinates': [-122.0862974...","{'type': 'Point', 'coordinates': [-122.0801222...",1.437596e+09,1.437597e+09,1


### Direct mongodb queries

You can also use direct mongodb queries during exploratory work. I do ask that you create a storage decorator (`emission/storage/decorations`) when you submit a pull request for ongoing use

In [33]:
import emission.core.get_database as edb

In [45]:
edb.get_timeseries_db().find_one()

{'_id': ObjectId('5bf5904f36a3f505a4f2fd4c'),
 'data': {'accuracy': 37.5,
  'altitude': 0,
  'filter': 'time',
  'fmt_time': '2015-07-22T08:14:53.881000-07:00',
  'heading': 0,
  'latitude': 37.3887767,
  'loc': {'coordinates': [-122.0876886, 37.3887767], 'type': 'Point'},
  'local_dt': {'day': 22,
   'hour': 8,
   'minute': 14,
   'month': 7,
   'second': 53,
   'timezone': 'America/Los_Angeles',
   'weekday': 2,
   'year': 2015},
  'longitude': -122.0876886,
  'sensed_speed': 0,
  'ts': 1437578093.881},
 'metadata': {'key': 'background/location',
  'platform': 'android',
  'read_ts': 0,
  'time_zone': 'America/Los_Angeles',
  'type': 'message',
  'write_fmt_time': '2015-07-22T08:14:54.840000-07:00',
  'write_local_dt': {'day': 22,
   'hour': 8,
   'minute': 14,
   'month': 7,
   'second': 54,
   'timezone': 'America/Los_Angeles',
   'weekday': 2,
   'year': 2015},
  'write_ts': 1437578094.84},
 'user_id': UUID('6a1717a2-99f9-4051-bb7a-d9c06f9287fa')}

In [47]:
# before running pipeline
edb.get_timeseries_db().distinct("metadata.key")

['background/location',
 'background/filtered_location',
 'statemachine/transition',
 'background/motion_activity']

In [72]:
# after running pipeline
edb.get_timeseries_db().distinct("metadata.key")

['background/location',
 'background/filtered_location',
 'statemachine/transition',
 'background/motion_activity',
 'stats/pipeline_time']

In [75]:
test = pd.DataFrame(list(edb.get_timeseries_db().find({"metadata.key": 'stats/pipeline_error'})))

In [78]:
edb.get_timeseries_db().find_one({"metadata.key": 'stats/pipeline_time'})

{'_id': ObjectId('5bf59ec936a3f508557a2470'),
 'data': {'name': 'USERCACHE',
  'reading': 0.016414622999946005,
  'ts': 1542823625.050756},
 'metadata': {'key': 'stats/pipeline_time',
  'platform': 'server',
  'time_zone': 'America/Los_Angeles',
  'write_fmt_time': '2018-11-21T10:07:05.051557-08:00',
  'write_local_dt': {'day': 21,
   'hour': 10,
   'minute': 7,
   'month': 11,
   'second': 5,
   'timezone': 'America/Los_Angeles',
   'weekday': 2,
   'year': 2018},
  'write_ts': 1542823625.051557},
 'user_id': UUID('6a1717a2-99f9-4051-bb7a-d9c06f9287fa')}

In [84]:
edb.get_usercache_db().find_one()

{'_id': ObjectId('5bf59fd36f0a85cfea3d6d69'),
 'data': [{'features': [{'geometry': {'coordinates': [-122.0876886,
       37.3887767],
      'type': 'Point'},
     'id': '5bf59ed936a3f508557a2681',
     'properties': {'exit_fmt_time': '2015-07-22T08:14:53.881000-07:00',
      'exit_local_dt': {'day': 22,
       'hour': 8,
       'minute': 14,
       'month': 7,
       'second': 53,
       'timezone': 'America/Los_Angeles',
       'weekday': 2,
       'year': 2015},
      'exit_ts': 1437578093.881,
      'feature_type': 'start_place',
      'raw_places': [ObjectId('5bf59ed336a3f508557a2474'),
       ObjectId('5bf59ed336a3f508557a2474')],
      'source': 'DwellSegmentationTimeFilter',
      'starting_trip': ObjectId('5bf59ed636a3f508557a24c0')},
     'type': 'Feature'},
    {'geometry': {'coordinates': [-122.0820411, 37.3920436], 'type': 'Point'},
     'id': '5bf59ed936a3f508557a2682',
     'properties': {'duration': 100.97699999809265,
      'ending_trip': ObjectId('5bf59ed636a3f508557a2

In [77]:
edb.get_timeseries_db().distinct("metadata.key")

['background/location',
 'background/filtered_location',
 'statemachine/transition',
 'background/motion_activity',
 'stats/pipeline_time']

In [71]:
edb.get_analysis_timeseries_db().distinct("metadata.key")

[]

Note that in this case, you need to know whether to use the `timeseries` or the `analysis_timeseries` collection

In [74]:
edb.get_analysis_timeseries_db().distinct("metadata.key")

['segmentation/raw_trip',
 'segmentation/raw_place',
 'segmentation/raw_section',
 'segmentation/raw_stop',
 'analysis/smoothing',
 'analysis/recreated_location',
 'analysis/cleaned_section',
 'analysis/cleaned_stop',
 'analysis/cleaned_place',
 'analysis/cleaned_trip',
 'inference/prediction',
 'analysis/inferred_section']

In [30]:
edb.get_analysis_timeseries_db().find_one({"metadata.key": 'analysis/cleaned_trip'})

{'_id': ObjectId('5bf59ed636a3f508557a24c0'),
 'data': {'distance': 25699.69350532018,
  'duration': 4120.635999917984,
  'end_fmt_time': '2015-07-22T09:23:34.517000-07:00',
  'end_loc': {'coordinates': [-122.0820411, 37.3920436], 'type': 'Point'},
  'end_local_dt': {'day': 22,
   'hour': 9,
   'minute': 23,
   'month': 7,
   'second': 34,
   'timezone': 'America/Los_Angeles',
   'weekday': 2,
   'year': 2015},
  'end_place': ObjectId('5bf59ed936a3f508557a2682'),
  'end_ts': 1437582214.517,
  'raw_trip': ObjectId('5bf59ed336a3f508557a2475'),
  'source': 'DwellSegmentationTimeFilter',
  'start_fmt_time': '2015-07-22T08:14:53.881000-07:00',
  'start_loc': {'coordinates': [-122.0876886, 37.3887767], 'type': 'Point'},
  'start_local_dt': {'day': 22,
   'hour': 8,
   'minute': 14,
   'month': 7,
   'second': 53,
   'timezone': 'America/Los_Angeles',
   'weekday': 2,
   'year': 2015},
  'start_place': ObjectId('5bf59ed936a3f508557a2681'),
  'start_ts': 1437578093.881},
 'metadata': {'key': '

In [10]:
ts.get_data_df("segmentation/raw_trip", time_query=None).columns

Index(['_id', 'distance', 'duration', 'end_fmt_time', 'end_loc',
       'end_local_dt_day', 'end_local_dt_hour', 'end_local_dt_minute',
       'end_local_dt_month', 'end_local_dt_second', 'end_local_dt_timezone',
       'end_local_dt_weekday', 'end_local_dt_year', 'end_place', 'end_ts',
       'metadata_write_ts', 'source', 'start_fmt_time', 'start_loc',
       'start_local_dt_day', 'start_local_dt_hour', 'start_local_dt_minute',
       'start_local_dt_month', 'start_local_dt_second',
       'start_local_dt_timezone', 'start_local_dt_weekday',
       'start_local_dt_year', 'start_place', 'start_ts', 'user_id'],
      dtype='object')

In [36]:
edb.get_analysis_timeseries_db().find({"user_id": test_user_id, "metadata.key": "analysis/cleaned_trip"}).count()

6

In particular, you can use this to access entries that are not in the timeseries

In [37]:
edb.get_uuid_db().distinct("uuid")

[UUID('de0f61b9-8e0d-45f0-8c64-09e6fbabd7e9')]

### Timeline

The trips and places maintain links to each other - e.g. `start_place`, `end_place`

In [ ]:
ct_df[["start_place", "end_place"]]

These are _primary key links_ to other entries in the database. It would be useful to have a doubly linked list representing this properly. The Timeline helps with that.

In [ ]:
import emission.storage.decorations.timeline as esdl

In [ ]:
trip_start_end_fuzz = 10 # seconds
ct_df = ts.get_data_df("analysis/cleaned_trip", time_query=None)
tl = esdl.get_cleaned_timeline(test_user_id, ct_df.iloc[0].start_ts - trip_start_end_fuzz, ct_df.iloc[-1].end_ts + trip_start_end_fuzz)

In [ ]:
for e in tl:
    if 'enter_ts' in e.data:
        # Must be place-like
        print(e.metadata.key, e.data.enter_fmt_time, "->", e.data.exit_fmt_time)
    else:
        print(e.metadata.key, e.data.start_fmt_time, "->", e.data.end_fmt_time)

In [ ]:
# The timeline is an iterator, so after it is consumed, it is empty
for e in tl:
    if 'enter_ts' in e.data:
        # Must be place-like
        print(e.metadata.key, e.data.enter_fmt_time, "->", e.data.exit_fmt_time)
    else:
        print(e.metadata.key, e.data.start_fmt_time, "->", e.data.end_fmt_time)

In [ ]:
stl = esdt.get_cleaned_timeline_for_trip(test_user_id, tl.first_trip().get_id())

In [ ]:
for e in stl:
    print(e.metadata.key)

In [ ]:
stl = esdt.get_cleaned_timeline_for_trip(test_user_id, tl.last_trip().get_id())

In [ ]:
for e in stl:
    print(e.metadata.key)

## Getting trip and section details ##

Once we have trip and section objects, we can retrieve the sensed data associated with them by querying for data in various streams that falls within the time ranges associated with the trip/section. Here again, our architecture of storing the analysis results as a separate datastream makes it easy to retrieve data at various levels of processing.

### Plot a processed trip or set of trips ###

In [ ]:
import emission.analysis.plotting.geojson.geojson_feature_converter as gfc
import emission.analysis.plotting.leaflet_osm.our_plotter as lo

In [ ]:
first_trip_for_user = ct_df.iloc[0]
first_trip_start_ts = first_trip_for_user.start_ts
first_trip_end_ts = first_trip_for_user.end_ts
trip_start_end_fuzz = 10 # seconds
trips_geojson_list = gfc.get_geojson_for_ts(test_user_id, first_trip_start_ts-trip_start_end_fuzz, ct_df.iloc[-1].end_ts+trip_start_end_fuzz)

In [ ]:
len(trips_geojson_list)

In [ ]:
map_list = lo.get_maps_for_geojson_trip_list(trips_geojson_list)

In [ ]:
len(map_list)

In [ ]:
map_list[0]

In [ ]:
map_list[-1]

In [ ]:
import branca.element as bre

In [ ]:
nrows = 2
ncols = 3
fig = bre.Figure()
for i, m in enumerate(map_list[:6]):
    fig.add_subplot(nrows,ncols,i+1).add_child(m)
fig

In [ ]:
nrows = 2
ncols = 3
fig = bre.Figure()
for i, map in enumerate(map_list[-6:]):
    fig.add_subplot(nrows,ncols,i+1).add_child(map)
fig

## Can you do better? ##


### Get locations with no processing, basic filtering and resampling for the first trip ###

In [ ]:
all_locs = ts.get_data_df("background/location",
                               time_query = esda.get_time_query_for_trip_like(
                                   "analysis/cleaned_section", ct_df.iloc[0]._id))

filtered_locs = ts.get_data_df("background/filtered_location",
                               time_query = esda.get_time_query_for_trip_like(
                                   "analysis/cleaned_section", ct_df.iloc[0]._id))

resampled_locs = ts.get_data_df("analysis/recreated_location",
                                 time_query = esda.get_time_query_for_trip_like(
                                   "analysis/cleaned_section", ct_df.iloc[0]._id))
print("Locations go from all = %d -> filtered = %d -> resampled = %d" % (len(all_locs),
                                                                         len(filtered_locs),
                                                                         len(resampled_locs)))

In [ ]:
all_locs[["_id", "latitude", "longitude", "fmt_time"]]

### Get the raw motion activity, in case you want to do different segmentation ###

In [ ]:
all_activity = ts.get_data_df("background/motion_activity",
                               esda.get_time_query_for_trip_like(
                                   "analysis/cleaned_section", ct_df.iloc[0]._id))

In [ ]:
all_activity.columns

In [ ]:
import emission.core.wrapper.motionactivity as ecwm

print("Found %d walking entries, %d on_foot entries" % (len(all_activity[all_activity.type == ecwm.MotionTypes.WALKING.value]), 
                                    len(all_activity[all_activity.type == ecwm.MotionTypes.ON_FOOT.value])))
print("Found %d motorized entries" % (len(all_activity[all_activity.type == ecwm.MotionTypes.IN_VEHICLE.value])))

### Plot the location points ###

In [ ]:
map_list = lo.get_maps_for_geojson_unsectioned([gfc.get_feature_list_from_df(all_locs),
                                                gfc.get_feature_list_from_df(filtered_locs),
                                                gfc.get_feature_list_from_df(resampled_locs)])

In [ ]:
fig = bre.Figure()
for i, map in enumerate(map_list):
    fig.add_subplot(1,3,i+1).add_child(map)
fig